In [3]:
import pandas as pd

In [24]:
def remove_outlier(df, col_name):
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3-q1
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df = df[(df[col_name] > fence_low) & (df[col_name] < fence_high)]
    return df

def RScoring(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

def FnMScoring(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

In [36]:
def give_products(df,key):
    df.sort_values(by='InvoiceDate')

    df=df.dropna(axis=0)
    df=df.drop_duplicates()

    df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
    #to convert the columns to days ,we need to get the last day cheaked in data
    last_date=df['InvoiceDate'].max()
    df['InvoiceDate']=last_date-df['InvoiceDate']
    df['InvoiceDate']=df['InvoiceDate'].dt.days

    df['the amount']=df['Quantity']*df['UnitPrice']

    r_df=df.groupby('CustomerID')['InvoiceDate'].min()
    f_df=df.groupby(df['CustomerID'])['InvoiceNo'].count()
    m_df=df.groupby(df['CustomerID'])['the amount'].sum()

    rf_data=pd.merge(r_df,f_df,on='CustomerID', how='inner')
    rfm_data=pd.merge(rf_data,m_df,on='CustomerID', how='inner')

    country_data=df.groupby(df['Country'])['CustomerID'].count()

    id_coun=df[['Country','CustomerID']]
    rfmc_data=pd.merge(rfm_data,id_coun,on='CustomerID', how='left')
    rfmc_data=rfmc_data.drop_duplicates()
    rfmc_data=rfmc_data.set_index('CustomerID')
    rfmc_data=rfmc_data.rename(columns={'InvoiceDate':'recency',
                                        'InvoiceNo':'frequency',
                                        'the amount':'monetary'})

    rfmc_data.groupby(rfmc_data['Country']).count()
    UK_rfmc_data=rfmc_data[rfmc_data['Country']=='India']
    
    L=0
    while L<=3:
        select_col=UK_rfmc_data.columns[L]
        remove_data=remove_outlier(UK_rfmc_data,select_col)
        UK_rfmc_data=remove_data
        # print('turn num ',L+1)
        # print(UK_rfmc_data.shape)
        L+=1
        if L==3:
            break

        #rfm score
    quantiles = UK_rfmc_data.quantile(q=[0.25,0.5,0.75])
    quantiles = quantiles.to_dict()



    UK_rfmc_data['R'] =UK_rfmc_data['recency'].apply(RScoring,
                                                args=('recency',quantiles,))
    UK_rfmc_data['F'] = UK_rfmc_data['frequency'].apply(FnMScoring,
                                                args=('frequency',quantiles,))
    UK_rfmc_data['M'] =UK_rfmc_data['monetary'].apply(FnMScoring,
                                                args=('monetary',quantiles,))


    UK_rfmc_data['RFM SCORE']=UK_rfmc_data[['R','F','M']].sum(axis=1)

    S_customer=UK_rfmc_data[UK_rfmc_data['RFM SCORE']==3]
    S_customer['RFM_Loyalty_Leve']='S'

    UK_rfmc_data=UK_rfmc_data[UK_rfmc_data['RFM SCORE']!=3]

    Loyalty_Level = ['A','B','C']
    Score_cuts = pd.qcut(UK_rfmc_data['RFM SCORE'], q = 3, labels = Loyalty_Level)
    UK_rfmc_data['RFM_Loyalty_Level'] = Score_cuts.values

    A_customer=UK_rfmc_data[UK_rfmc_data['RFM_Loyalty_Level']=='A']
    B_customer=UK_rfmc_data[UK_rfmc_data['RFM_Loyalty_Level']=='B']
    C_customer=UK_rfmc_data[UK_rfmc_data['RFM_Loyalty_Level']=='C']

    merged_df = pd.merge(df, UK_rfmc_data[['R', 'F', 'M']], left_on='CustomerID', right_index=True, how='left')

    # Define shelf capacities
    shelf_capacities = [100, 200, 300, 400, 500]

    # Sort products by RFM scores
    merged_df = merged_df.sort_values(by=['R', 'F', 'M'], ascending=False)

    # Initialize a dictionary to store shelf allocations
    shelf_allocations = {}

    # Initialize a set to keep track of allocated products
    allocated_products = set()

    # Iterate through products and allocate to shelves
    for index, row in merged_df.iterrows():
        for shelf_id, capacity in enumerate(shelf_capacities):
            if row['Description'] not in allocated_products and row['Quantity'] <= capacity:
                shelf_allocations.setdefault(shelf_id, []).append(row['Description'])
                shelf_capacities[shelf_id] -= row['Quantity']
                allocated_products.add(row['Description'])
                break

    # Print shelf allocations
    for shelf_id, products in shelf_allocations.items():
        print(f"Shelf {shelf_id+1}: {', '.join(products)}")
    
    # print('Products : ',products)

    # lowest RF scores = STOCK CLEARANCE
    merged_df['RF_Sum'] = merged_df['R'] + merged_df['F']

    merged_df_sorted = merged_df.sort_values(by='RF_Sum', ascending=True)
    lowest_rf_product = merged_df_sorted.iloc[0]

    """ print("Product with the lowest RF score:")
    print(lowest_rf_product)
    print(lowest_rf_product.Description) """

    # highest MF scores = PRODUCT OF THE MONTH
    merged_df['MF_Sum'] = merged_df['M'] + merged_df['F']

    merged_df_sorted2 = merged_df.sort_values(by='MF_Sum', ascending=False)
    highest_mf_product = merged_df_sorted2.iloc[0]

    """ print("Product with the highest MF score:")
    print(highest_mf_product)
    print(highest_mf_product.Description) """

    # highest MF scores = SEASONAL
    merged_df['RM_Sum'] = merged_df['R'] + merged_df['M']

    merged_df_sorted3 = merged_df.sort_values(by='RM_Sum', ascending=False)
    highest_rm_product = merged_df_sorted3.iloc[0]

    """ print("Product with the highest RM score:")
    print(highest_rm_product)
    print(highest_rm_product.Description) """

    rmf = (merged_df['R'] + merged_df['M'] + merged_df['F']).mean()

    # print()

    return {
        'Category':key,
        'Lowest RF Product' : lowest_rf_product.Description,
        'Highest MF Product' : highest_mf_product.Description,
        'Highest RM Product' : highest_rm_product.Description,
        'RMF Value' : rmf,
        'Items' : products
    }

In [39]:
dataset = {
    'Bathroom Supplies' : 'datasets/bathroom.csv',
    'Beauty' : 'datasets/beauty.csv',
    'Clothing' : 'datasets/clothing.csv',
    'Electronics' : 'datasets/electronics.csv',
    'Grocery' : 'datasets/grocery.csv',
    'Home Decor' : 'datasets/homedecor.csv',
    'Medical' : 'datasets/medical.csv',
    'Packaged' : 'datasets/packaged.csv',
    'Refrigerated' : 'datasets/refrigerated.csv',
    'Sporting' : 'datasets/sporting.csv',
    'Stationary' : 'datasets/stationary.csv'
}


In [40]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')
for k,v in dataset.items():
    df=pd.read_csv(v)
    data = give_products(df,k)
    print(data)
    print()

Shelf 1: Plunger, Hand Towels, Colgate Toothpaste, Hand Soap, Colgate Toothbrush, Shower Curtain, Trash Can, Air Freshener, Toilet Paper, Mirror Cleaner, Body Wash, Colgate Dental Floss, Cotton Swabs, Shampoo, Tissues, Colgate Mouthwash, Shaving Cream, Shower Mat, Conditioner, Toilet Brush
{'Category': 'Bathroom Supplies', 'Lowest RF Product': 'Shower Curtain', 'Highest MF Product': 'Plunger', 'Highest RM Product': 'Plunger', 'RMF Value': 7.450566759485838, 'Items': ['Plunger', 'Hand Towels', 'Colgate Toothpaste', 'Hand Soap', 'Colgate Toothbrush', 'Shower Curtain', 'Trash Can', 'Air Freshener', 'Toilet Paper', 'Mirror Cleaner', 'Body Wash', 'Colgate Dental Floss', 'Cotton Swabs', 'Shampoo', 'Tissues', 'Colgate Mouthwash', 'Shaving Cream', 'Shower Mat', 'Conditioner', 'Toilet Brush']}

Shelf 1: Eyeliner, Foundation, Body Lotion, Mascara, Eyeshadow, Moisturizer, Nail Polish, Makeup Remover, Lipstick, Blush
{'Category': 'Beauty', 'Lowest RF Product': 'Lipstick', 'Highest MF Product': 'Ey